# __Tweepy: Busca de tweets com Twitter API V2__

Para a API V2 do Twitter temos dois _end points_ de busca de tweets:

    1. Busca recente, onde podemos coletar tweets dentro de uma janela de 7 dias;
    2. Busca completa, onde podemos coletar tweets desde de março de 2006

In [18]:
import tweepy, os, requests
import pandas as pd
import tweepy as tw
from datetime import datetime, timezone

In [10]:
keys = {
        'consumer_key': os.environ.get('API_KEY'),
        'consumer_secret': os.environ.get('API_SECRET'),
        'access_token_key': os.environ.get('ACCESS_TOKEN'),
        'access_token_secret': os.environ.get('ACCESS_TOKEN_SECRET'),
        'bearer_token': os.environ.get('BEARER_TOKEN')
    }

Para a API V2 não é necessário criar um autenticador, basta criar uma instância de _Client_ do Tweepy e colocar as seguintes informaçães para realizar a autenticação.

In [232]:
client = tw.Client(
    bearer_token=keys['bearer_token'],
    #keys['consumer_key'],
    #keys['consumer_secret'],
    #keys['access_token_key'],
    #keys['access_token_secret'],
    wait_on_rate_limit=True,
    #return_type=dict
)

## Busca Recente

In [186]:
tweets_recent = client.search_recent_tweets(
    query='python lang:pt', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency) 
)

In [188]:
tweets = [tweet for tweet in tweets_recent['data']]
len(tweets)

49

In [189]:
tweets[0]

{'id': '1567301404192120839',
 'text': 'O meu professor segue firme tentando obrigar a turma a aprender Fortran, e disse que vai zerar se alguém entregar em Python'}

Mas só recupera esses dados do Tweet????

Para adicionar mais campos é necessário especificar na busca.

In [190]:
tweets_recent = client.search_recent_tweets(
    query='python lang:pt', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency)
    expansions=['author_id', 'geo.place_id', 'entities.mentions.username', 'referenced_tweets.id', 'in_reply_to_user_id', 'referenced_tweets.id.author_id'],
    place_fields=['full_name', 'country'],
    tweet_fields=['id', 'text', 'author_id', 'conversation_id', 'created_at', 'entities', 'in_reply_to_user_id', 'lang',
                   'public_metrics', 'referenced_tweets'])

tweets = [tweet for tweet in tweets_recent['data']]
len(tweets)

49

In [192]:
tweets[0]

{'conversation_id': '1567301404192120839',
 'author_id': '1078692303273893890',
 'created_at': '2022-09-06T23:59:08.000Z',
 'id': '1567301404192120839',
 'text': 'O meu professor segue firme tentando obrigar a turma a aprender Fortran, e disse que vai zerar se alguém entregar em Python',
 'lang': 'pt',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0}}

In [193]:
tweets_recent.keys() # informações coletadas

dict_keys(['data', 'includes', 'errors', 'meta'])

In [194]:
tweets_recent['includes'].keys()

dict_keys(['users', 'tweets'])

In [195]:
tweets_recent['includes']['users'][0]

{'id': '1078692303273893890',
 'name': 'derrotada da Unicamp',
 'username': 'feercrystal'}

In [196]:
tweets_recent['includes']['tweets'][0]

{'conversation_id': '1567184942525779969',
 'author_id': '1266799817096904705',
 'created_at': '2022-09-06T17:57:23.000Z',
 'id': '1567210368107253761',
 'entities': {'mentions': [{'start': 0,
    'end': 13,
    'username': 'LAMI01773671',
    'id': '1238884889124122626'}]},
 'referenced_tweets': [{'type': 'replied_to', 'id': '1567204149263048705'}],
 'text': '@LAMI01773671 tudo bem eu já desejo as vezes // vc faz?',
 'lang': 'pt',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 1,
  'like_count': 1,
  'quote_count': 0},
 'in_reply_to_user_id': '1238884889124122626',
 'geo': {'place_id': '7cd48c674d4c69f7'}}

In [197]:
tweets_recent['meta']

{'newest_id': '1567301404192120839',
 'oldest_id': '1567263831067398145',
 'result_count': 49,
 'next_token': 'b26v89c19zqg8o3fpz8mu60m1ez3ups0mmlte9kdbn4hp'}

### Utilizando Paginação para Coleta de Mais Tweets

In [233]:
tweets = []

for tweet in tw.Paginator(
    client.search_recent_tweets,
    query='programação', # query de busca (lang:pt é um filtro da própria busca do tweet)
    end_time='2022-09-07T00:00:00Z', # procura tweets antes desse dia
    start_time='2022-09-03T00:00:00Z', # procura tweets após esse dia
    max_results = 50, # número máximo de tweets recuperados (o máximo continua sendo 100 tweets por página)
    sort_order='recency', # ordenação dos tweets, pode ser por relevância (relevancy) ou tweets mais recentes (recency)
    expansions=['author_id', 'geo.place_id', 'entities.mentions.username', 'referenced_tweets.id', 'in_reply_to_user_id', 'referenced_tweets.id.author_id'],
    place_fields=['full_name', 'country'],
    tweet_fields=['id', 'text', 'author_id', 'conversation_id', 'created_at', 'entities', 'in_reply_to_user_id', 'lang',
                   'public_metrics', 'referenced_tweets']).flatten(limit=150):
    tweets.append(tweet)

len(tweets)

150

In [240]:
tweets[0].data

{'public_metrics': {'retweet_count': 3218,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'id': '1567301618374266886',
 'text': 'RT @leandroruschel: A única mídia tradicional com linha editorial mais alinhada ao presidente corre o risco de ser retirada do ar por 24hs,…',
 'created_at': '2022-09-06T23:59:59.000Z',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1567281859062632450'}],
 'conversation_id': '1567301618374266886',
 'lang': 'pt',
 'author_id': '49828116',
 'entities': {'mentions': [{'start': 3,
    'end': 18,
    'username': 'leandroruschel',
    'id': '174818242'}]}}